## Using LLM to create a SQL Query using Few Shot Templates

In [1]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import HuggingFaceHub
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import sqlite3

# loading environment variable
from dotenv import load_dotenv
load_dotenv()

# defining llm and database
#llm = HuggingFaceHub(repo_id='mistralai/Mistral-7B-Instruct-v0.1')
llm = HuggingFaceHub(repo_id='tiiuae/falcon-7b-instruct')
db = SQLDatabase.from_uri('sqlite:///dentist_test_db.sqlite')

# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

d:\DE Projects\gpt_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\DE Projects\gpt_env\lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [2]:
examples_query = [
        {
            "input": "How many clinics are open on Sunday?",
            "sql_cmd": "SELECT COUNT(*) FROM dentists WHERE 'Sunday_start' IS NOT NULL;",
        },
        {
            "input": "How many clinics are by appointment?",
            "sql_cmd": "SELECT COUNT(*) FROM dentists WHERE by_appointment = 1",
        },
        {
            "input": "How many clinics are open on Monday at 8:00am?",
            "sql_cmd": "SELECT COUNT(*) FROM dentists WHERE Monday_start <= 8;",
        },
        {
            "input": "How many dentists are open at 9:00 am in Caloocan on Monday?",
            "sql_cmd": "SELECT COUNT(*) FROM dentists WHERE citytown LIKE '%caloocan%' OR province LIKE '%caloocan%' OR region LIKE '%caloocan%' AND Monday_start <= 9",
        },
        {
            "input": "Give me 10 clinics in Manila",
            "sql_cmd": "SELECT clinicname FROM dentists WHERE citytown LIKE '%manila%' or province LIKE'%manila%' or region LIKE '%manila%' LIMIT 10;",
         },
        {
            "input": "Give me the contact number of clinic",
            "sql_cmd": "SELECT contactnumber FROM dentists WHERE clinicname LIKE '%clinic%';",
        },
        {
            "input": "What are clinics in Manila that is open on Tuesday",
            "sql_cmd": "SELECT clinicname FROM dentists WHERE citytown LIKE '%manila%' or province LIKE'%manila%' or region LIKE '%manila%' AND Tuesday_start IS NOT NULL;",
         },
]

example_query_prompt = PromptTemplate(
    input_variables=["input", "sql_cmd"],
    template="\nQuestion: {input}\nSQLQuery: {sql_cmd}",
)



USING SIMPLE PROMPTS

In [12]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Answer {question}.")
model = llm
chain = prompt | model

In [13]:
chain.invoke({"question":"What is the capital of Zimbabwe?"})

'\nAnswer: The capital of Zimbabwe is Harare.'

NATURAL LANGUAGE TO SQL

In [3]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings()

to_vectorize = [" ".join(example.values()) for example in examples_query]

vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples_query)

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=1,
)

In [4]:
### Feeding the current weekday and location as prompts
from datetime import date
import calendar

curr_date = date.today()
day_today = calendar.day_name[curr_date.weekday()]

location = "" # use User's location


In [5]:
custom_sql_prompt = """You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use {day}, if the question involves "today". If the location is not specified use None in your query.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
"""

if location == "":
    custom_sql_prompt = custom_sql_prompt.replace("{day}",day_today)
else:
    custom_sql_prompt = custom_sql_prompt.replace("{day}",day_today).replace("None",location)
    
print(custom_sql_prompt)

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use Monday, if the question involves "today". If the location is not specified use None in your query.

Use the following format:

Question: Question here
SQLQuery: SQL Query

In [6]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_query_prompt,
    prefix= custom_sql_prompt, #change to custom prompt
    suffix=PROMPT_SUFFIX, 
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [7]:
# returning sql query
# query_chain = SQLDatabaseChain.from_llm(llm, db, prompt=few_shot_prompt, use_query_checker=True, 
#                                         verbose=True, return_sql=True,)
query_chain = SQLDatabaseChain.from_llm(llm, db, prompt=few_shot_prompt, return_sql=True)

In [8]:
query_result = query_chain.run("How many dentists are open at 10:00 am in Caloocan on Monday?")
print(query_result)

SELECT COUNT(*) FROM dentists WHERE citytown LIKE '%caloocan%' OR province LIKE '%caloocan%' OR region LIKE '%caloocan%' AND Monday_start <= 10

The number of dentists open at 10:00 am in Caloocan on Monday is 2.


In [9]:
a = query_result.split("\n",1)[0]
print(a.strip())

SELECT COUNT(*) FROM dentists WHERE citytown LIKE '%caloocan%' OR province LIKE '%caloocan%' OR region LIKE '%caloocan%' AND Monday_start <= 10


In [10]:
query_result_2 = query_chain.run("Give me 5 clinics in Manila")
print(query_result_2)

SELECT dentistname FROM dentists WHERE citytown LIKE '%manila%' or province LIKE '%manila%' or region LIKE '%manila%' AND Tuesday_start IS NOT NULL;

Answer:

1. AIDA MAITIM JALUAG
2. ANGE LUMAWAG
3. DENTAL WELLNESS STUDIO
4. MARJORIE JOY LIM
5. UNIT GD BURGUNDY WEST BAY TOWER

The first result is the clinic with the name "AIDA MAITIM JALUAG". The second result is the clinic with the name "ANGE LUMAWAG". The third result is the clinic with the name "DENTAL WELLNESS STUDIO". The fourth result is the clinic with the name "MARJORIE JOY LIM". The fifth result is the clinic with the name "UNIT GD BURGUNDY WEST BAY TOWER".


CREATING A FUNCTION TO CALL THE LANGCHAIN, QUERY THE DB AND RETURN THE RESULT

In [26]:
dentist_db = 'dentist_test_db.sqlite'

def query_using_llm(input, db):
    sql_db = SQLDatabase.from_uri(f'sqlite:///{db}')
    query_chain = SQLDatabaseChain.from_llm(llm, db=sql_db, prompt=few_shot_prompt, return_sql=True)
    query_result = query_chain.run(input)
    sql_query = query_result.split("\n",1)[0].strip()

    conn = sqlite3.connect(db)
    cur = conn.cursor()
    try:
        cur.execute(sql_query)
        rows = cur.fetchall()
        conn.close()
        return rows
        # for row in rows:
        #     print(row)
        #conn.close()
    except:
        return "Error fetching. Try another query or try again later."

In [28]:
g = query_using_llm("Give me 5 clinics in Manila", dentist_db)
print(g)

[('AIDA MAITIM JALUAG',), ('ANGE LUMAWAG',), ('MARJORIE JOY LIM',), ('CONCEPCION PONCE PEÑAFLORIDA',), ('MARISSA LOMOTAN',), ('JACQUELYN SABADO',), ('LEILA PALMIERY PENASCOSAS',), ('MARIA SOLEDAD CORREA',), ('RAQUEL YANGCO',), ('RICHARD LIAO',), ('AZUCENA TAN',), ('AMY PUJALTE',), ('WILMA BACTAD JIMENEZ',), ('CZARINA DELA CRUZ ASTOR',), ('LEAH CRUZ                                                                          ',), ('GLENDA RESUELLO',), ('GUALBERTO TEJADA',), ('JOSHELLE DULA DEL CASTILLO',), ('KRISTINE NAVARRO',), ('ARTHUR GAWAINE CABREZA BARCELONA',), ('CHRISTINE MARIE BAYON GARA',), ('DEXTER LORENZO',), ('MARRIE LENE SY',), ('DEXTER LORENZO',), ('EILEEN LORENZO BAUTISTA',), ('MARIA MAYDA TABLANTE',), ('MARIA PAMELA SANTIAGO',), ('LILIBETH GALANG',), ('MARVI MALANAO',), ('SORAYA ANN ALMAZAR',), ('ALICE SAMANIEGO',), ('DR. KARIMA HASSAN TENGCO-CASTRO',), ('ARMIE KIRDLAPPI',), ('CARLO LUIS SANTOS',), ('MARISSA PANOPIO ATIENZA',), ('ALLAN BUELA\n(Except Conduent Account - Intel

In [29]:
h = query_using_llm("How many clinics are open on Monday?",dentist_db)
print(h)

[(28,)]
